<center>
    <tr>
    <td><img src="images/Quansight_Logo_Lockup_1.png" width="25%"></img></td>
    </tr>
</center>

# Filter Banks & Texture Analysis

+ Basic approach: apply *filter banks*—sequence of pre-determined filters (kernels) of varying characteristics—to signals (images)
+ Permits encoding of images in *feature vectors* of fixed length
+ Applied, e.g., to [texture classification](https://www.robots.ox.ac.uk/~vgg/research/texclass/filters.html)

## The Leung-Malik (LM) Filter Bank

The LM set is a multi-scale, multi-orientation filter bank with 48 filters. It consists of first and second derivatives of Gaussians at 6 orientations and 3 scales making a total of 36; 8 Laplacian of Gaussian (LOG) filters; and 4 Gaussians. 

LM  Small (LMS) filters occur at scales $\sigma = \{1, \sqrt{2}, 2, 2 \sqrt{2} \}$.  The first and second derivates occur at the first three scales with an elongation factor of 3 (i.e., $\sigma_x = \sigma$ and $\sigma_y = 3 \sigma$).  The Gaussians occuer at four basic scales.  The 8 LOG occur at $\sigma$ and $3 \sigma$.

<br/>
<center>
    <tr>
    <td><img src="images/lmfilters.jpg" width="75%"></img></td>
    </tr><br/>
    Figure from https://www.robots.ox.ac.uk/~vgg/research/texclass/filters.html.</center>
<br/>

LM Large (LML) filters occur at scales $\sigma = \{ \sqrt{2},2,2\sqrt{2},4 \}$.

### Constructing the LM Filter Bank

In [ ]:
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import scipy as sp
from scipy import signal
from scipy import io
from scipy.spatial import distance

In [ ]:
F = sp.io.loadmat('data/lm.mat') # Load data from Matlab .mat format
# ... examine contents of F

In [ ]:
filter_bank = F['LM']
nr = 4
nc = 48//nr
plt.figure(figsize=(14,5))
plt.suptitle('Leung-Malik Filter Bank')
for i in range(48):
    plt.subplot(nr, nc, i+1)
    fig = plt.imshow(filter_bank[:,:,i], cmap='gray')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
plt.show()    

In [ ]:
feature_vectors = np.empty((4,48))  # We will use this to store feature vectors.

In [ ]:
filenames = [
    'data/textures/banded_0023.jpg',
    'data/textures/interlaced_0201.jpg',
    'data/textures/knitted_0204.jpg',
    'data/textures/lined_0177.jpg', 
    'data/textures/sprinkled_0144.jpg', 
    'data/textures/studded_0217.jpg', 
    'data/textures/woven_0131.jpg', 
    'data/textures/zigzagged_0133.jpg',
    'data/textures/matted_0166.jpg'
]

In [ ]:
# Examine a single file from the list
f_idx = 1

im = cv.imread(filenames[f_idx], 0)
print(im.shape)

plt.figure(figsize=(5,5))
plt.title(filenames[f_idx])
plt.imshow(im, cmap='gray');

In [ ]:
# Allocate space to store convolutions of all filters with this image
w, h = im.shape
_, _, num_filters = filter_bank.shape
responses = np.empty([w, h, num_filters])
print(responses.shape)

In [ ]:
for i in range(num_filters):
    responses[:,:,i] = sp.signal.convolve(im, filter_bank[:,:,i], mode='same')

In [ ]:
plt.figure(figsize=(14,5))
plt.suptitle('LM filter responses')
for i in range(48):
    plt.subplot(nr, nc, i+1)
    fig = plt.imshow(responses[:,:,i], cmap='gray')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
plt.show() 

In [ ]:
# Prepare feature vectors from each convolution:
# means & standard deviations of each response
mean_fv = np.mean(responses,(0,1)).flatten()
std_fv = np.std(responses,(0,1)).flatten()
feature_vector = np.hstack((mean_fv, std_fv))
print(f'{feature_vector.shape}')

In [ ]:
# View feature vector for this image as bar plot
plt.figure(figsize=(10,5))
plt.bar(np.arange(len(feature_vector)), feature_vector, color='red')
plt.title(f'{filenames[f_idx]}');

In [ ]:
# Encode all previous steps into a function that loops over all files
# This time, feature vectors store means only
def make_responses(filter_bank, filenames):
    _, _, num_filters = filter_bank.shape        
    num_files = len(filenames)

    feature_vectors = np.empty((num_files, num_filters))
    
    for i in range(num_files):
        filename = filenames[i]
        print(f'Processing {filename}')

        im = cv.imread(filename, 0)
        for j in range(num_filters):
            feature_vectors[i,j] = np.mean(sp.signal.convolve(im, filter_bank[:,:,j], mode='same'))
    
    return feature_vectors

In [ ]:
feature_vectors = make_responses(filter_bank, filenames)
print(feature_vectors.shape)

In [ ]:
x = np.arange(48)

plt.figure(figsize=(10,5))
for k,c in enumerate(list('rbgc')):
    plt.bar(x, feature_vectors[k,:], color=c, alpha=0.3, label=f'{filenames[k]}')
plt.legend()
plt.title('Overlay of feature vectors (4 files)');

In [ ]:
pairwise_distances = distance.pdist(feature_vectors, 'euclidean')
print(pairwise_distances.shape) # Returns upper triangle of symmetric matrix as a 1D array
print(pairwise_distances) # Upper triangular portion of symmetric matrix

In [ ]:
# Convert to matrix & pretty print as DataFrame
D = distance.squareform(pairwise_distances)
pd.DataFrame(data=D, index=filenames, columns=filenames)

**Take-home message**: Encoding images with filter banks yields feature vectors for convenient comparison.
+ Used for, e.g., texture analysis
+ Condenses images of arbitrary size to small number of numerical features
+ Form of *feature engineering* in machine learning (e.g., before classification)
+ Distinct choices of distance metrics & filter banks are possible

---

## The Schmid (S) Filters

The S set consists of 13 rotationally invariant filters of the form

$$
F(r, \sigma, \tau) = 
F_0 (\sigma, \tau) + \cos \left( \frac{\pi \tau r}{\sigma} \right) e ^ { - \frac{r^2}{2 \sigma^2} }
$$

Schmid Filter Bank equation
where $F_{0}$ is added to obtain a zero DC component with the  $(\sigma, \tau)$ pair taking values $(2,1)$, $(4,1)$, $(4,2)$, $(6,1)$, $(6,2)$, $(6,3)$, $(8,1)$, $(8,2)$, $(8,3)$, $(10,1)$, $(10,2)$, $(10,3)$ and $(10,4)$. The filters are shown below. 

<br/>
<center>
    <tr>
    <td><img src="images/csfilters.jpg" width="75%"></img></td>
    </tr><br/>
    Figure from https://www.robots.ox.ac.uk/~vgg/research/texclass/filters.html.</center>
<br/>

All the filters have rotational symmetry.

We can repeat the process above to analyze the same images with this filter bank.

In [ ]:
# Load Schmid Filter Bank from Matlab .mat file
F = sp.io.loadmat('data/s.mat')
filter_bank = F['S']
print(filter_bank.shape)

In [ ]:
filter_bank.sum(axis=(0,1)) # Should all be approx. zero (DC component)

In [ ]:
# Visualize the filter bank
nr = 2
nc = 7
plt.figure(figsize=(14,5))
plt.suptitle('Schmid Filter Bank')
for i in range(13):
    plt.subplot(nr, nc, i+1)
    fig = plt.imshow(filter_bank[:,:,i], cmap='gray')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
plt.show()    

In [ ]:
feature_vectors_schmid = make_responses(filter_bank, filenames)
print(feature_vectors_schmid.shape)

In [ ]:
x = np.arange(len(feature_vectors_schmid[0]))

plt.figure(figsize=(10,5))
for k,c in enumerate(list('rbgc')):
    plt.bar(x, feature_vectors_schmid[k,:], color=c, alpha=0.3, label=f'{filenames[k]}')
plt.legend()
plt.title('Feature vectors (4 files, S bank)');

In [ ]:
pairwise_distances = distance.pdist(feature_vectors_schmid, 'euclidean')
print(pairwise_distances.shape) # Returns upper triangle of symmetric matrix as a 1D array
print(pairwise_distances) # Upper triangular portion of symmetric matrix

In [ ]:
# Convert to matrix & pretty print as DataFrame
D = distance.squareform(pairwise_distances)
pd.DataFrame(data=D, index=filenames, columns=filenames)

---

## The Maximum Response (MR) Filter Banks 

Each of the reduced MR sets is derived from a common Root Filter Set (RFS) which consists of 38 filters and is very similar to LM. The filters used in the RFS bank are a Gaussian and a Laplacian of Gaussian both with $\sigma=10$ pixels (these filters have rotational symmetry), an edge filter at 3 scales (scale values) = $\{(1,3), (2,6), (4,12)\}$ and a bar filter at the same 3 scales. The latter two filters are oriented and, as in LM, occur at 6 orientations at each scale. The filter bank is shown below.

<br/>
<center>
    <tr>
    <td><img src="images/mr8filters.jpg" width="75%"></img></td>
    </tr><br/>
    Figure from https://www.robots.ox.ac.uk/~vgg/research/texclass/filters.html.</center>

In [ ]:
F = sp.io.loadmat('data/rfs.mat')
print(F['RFS'].shape)

filter_bank_mr = F['RFS']
nr = 3

nc = 13
plt.figure(figsize=(14,4))
plt.suptitle('Maximum Response (MR) Filter Bank')
for i in range(38):
    plt.subplot(nr, nc, i+1)
    fig = plt.imshow(filter_bank_mr[:,:,i], cmap='gray')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
plt.show()    

In [ ]:
feature_vectors_mr = make_responses(filter_bank_mr, filenames)

In [ ]:
x = np.arange(len(feature_vectors_mr[0]))

plt.figure(figsize=(10,5))
for k,c in enumerate(list('rbgc')):
    plt.bar(x, feature_vectors_mr[k,:], color=c, alpha=0.3, label=f'{filenames[k]}')
plt.legend()
plt.title('Feature vectors (4 files, MR bank)');

In [ ]:
pairwise_distances = distance.pdist(feature_vectors_mr, 'euclidean')
print(pairwise_distances.shape) # Returns upper triangle of symmetric matrix as a 1D array
print(pairwise_distances) # Upper triangular portion of symmetric matrix

In [ ]:
# Convert to matrix & pretty print as DataFrame
D = distance.squareform(pairwise_distances)
pd.DataFrame(data=D, index=filenames, columns=filenames)

---
Based on materials from Prof. Faisal Qureshi (Faculty of Science, Ontario Tech University, Oshawa ON, Canada, http://vclab.science.ontariotechu.ca)

<center>
    <tr>
    <td><img src="images/Quansight_Logo_Lockup_1.png" width="25%"></img></td>
    </tr>
</center>